In [1]:
import warnings
from loguru import logger

warnings.simplefilter(action='ignore', category=FutureWarning)

from edgedroid.models.timings import *
import edgedroid.data as default_data

logger.enable("edgedroid")

data = default_data.load_curve_fitting_data()
data

In [2]:
import numpy as np

data["outlier"] = False
for _, df in data.groupby(["prev_ttf", "prev_duration"]):
    outliers = ((df["exec_time"] > np.percentile(df["exec_time"], 95)) | (df["exec_time"] < np.percentile(df["exec_time"], 5)))
    outlier_idx = df.loc[outliers].index
    data.loc[outlier_idx, "outlier"] = True
data

In [3]:
from collections import deque
import pandas as pd
from typing import Tuple, Deque
from tqdm.notebook import tqdm

result_rows = deque()
for participant in tqdm(data["participant"].unique(), desc="Run"):
    class MultiCurveFittingTestModel(MultiCurveFittingExecutionTimeModel):
        @staticmethod
        def get_data() -> pd.DataFrame:
            data = MultiCurveFittingExecutionTimeModel.get_data()
            return data[data["participant"] != participant].copy()
        
        def __init__(self, neuroticism: float):
            super().__init__(
                neuroticism=neuroticism,
            )
            
    class Legacy(LegacyModel):
        @staticmethod
        def get_data() -> pd.DataFrame:
            data = MultiCurveFittingExecutionTimeModel.get_data()
            return data[data["participant"] != participant].copy()
        
        
    class PowerFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (PowerFit,)
        
    class SquareFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (SquareFit,)
        
    class CubeFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (CubeFit,)
        
    class ExponentialFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (ExponentialFit,)
        
    def get_test_data(*args, **kwargs) -> (
        Tuple[
            pd.DataFrame,
            pd.arrays.IntervalArray,
            pd.arrays.IntervalArray,
            pd.arrays.IntervalArray,
        ]
    ):
        data, *rest = ExecutionTimeModel.get_data()
        return (data[data["run_id"] != participant].copy(), *rest)

    class NaiveTestModel(FirstOrderETM):
        get_data = get_test_data
        
    class NaiveAggTestModel(FirstOrderAggregateETM):
        get_data = get_test_data
            
    run_data = data[data["participant"] == participant].copy()

    neuroticism = run_data["neuroticism"].unique()[0]
    naive_model = NaiveTestModel()
    naive_mean_model = NaiveAggTestModel(np.mean)
    naive_median_model = NaiveAggTestModel(np.median)
    
    power_fit_model = PowerFitTestModel(neuroticism)
    square_fit_model = SquareFitTestModel(neuroticism)
    cube_fit_model = CubeFitTestModel(neuroticism)
    exponential_fit_model = ExponentialFitTestModel(neuroticism)
    multi_fit_model = MultiCurveFittingExecutionTimeModel(neuroticism)
    
    legacy_model = Legacy()
    
    # deque containing (model name, model object, ttf_bins, window_size, kernel name)
    models: Deque[Tuple[str, ExecutionTimeModel, int, int, str]] = deque()
    models.extend((
        ("1st order", naive_model, 0, 0, "none"),
        ("1st order (median)", naive_median_model, 0, 0, "none"),
        ("1st order (mean)", naive_mean_model, 0, 0, "none"),
        ("a * x^b + c", power_fit_model , 0, 0, "none"),
        ("a * x^2 + b * x + c", square_fit_model, 0, 0, "none"),
        ("a * x^3 + b * x^2 + c * x + d", cube_fit_model, 0, 0, "none"),
        ("a * e^x + b", exponential_fit_model, 0, 0, "none"),
        ("multi curve", multi_fit_model, 0, 0, "none"),
        ("legacy", legacy_model, 0, 0, "none")
    ))
        
    for _ in range(1):
        for i, (prev_ttf, exec_time, neuro, prev_duration, outlier) in enumerate(run_data[["prev_ttf", "exec_time", "neuro", "prev_duration", "outlier"]].itertuples(index=False)):
            for model_name, model, ttf_bins, window_size, kernel_name in models:
                if i == 0:
                    # reset the model between runs
                    model.reset()
                else:
                    # first row has no previous ttf
                    model.advance(prev_ttf)
                    
                prediction = model.get_execution_time()
                error = np.nan if outlier else (prediction - exec_time)
                result_rows.append({
                    "real": exec_time,
                    "prediction": prediction,
                    "error": error,
                    "sqr_error": np.square(error),
                    "model": model_name,
                    "duration": prev_duration,
                    "neuro": neuro,
                    # "kernel": kernel_name,
                    # "ttf_bins": ttf_bins,
                    # "window_size": window_size,
                })

results = pd.DataFrame(result_rows)
for col in ("model", ):  # "kernel"):
    results[col] = results[col].astype("category")

results.to_parquet("./full_validation_ignore_outliers.parquet")
results


# remove outliers from curves
# include old edgedroid
# choose the best models and carry this forward - experimentation, evaluation
# constant ttfs
# some variations
# maybe qdisc
# autoregressive process on the delays

In [4]:
results.groupby(["model", "neuro", "duration"], observed=True)["sqr_error"].mean().sort_values(ascending=True)

In [5]:
results.groupby(["model"], observed=True)["sqr_error"].describe().sort_values(by="mean", ascending=True)